In [1]:
import pandas as pd
from pysus.online_data.SIA import SIA
from datetime import datetime
import os
import gc

# Configurações
ano = 2023
grupos = ['PA']
output_dir = "/home/amcjr/pysus/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

log_file = os.path.join(output_dir, f"log_processo_{datetime.now().strftime('%d%m%y_%H%M%S')}.xlsx")

def iniciar_log():
    df_log = pd.DataFrame(columns=["Inicio", "Fim", "Duracao (horas)", "Arquivos Processados"])
    return df_log

def registrar_log(df_log, inicio, fim, arquivos_processados):
    duracao = (fim - inicio).total_seconds() / 3600.0
    duracao_str = f"{int(duracao)}:{int((duracao - int(duracao)) * 60)}"
    novo_registro = pd.DataFrame([{
        "Inicio": inicio.strftime('%d-%m-%y %H:%M:%S'),
        "Fim": fim.strftime('%d-%m-%y %H:%M:%S'),
        "Duracao (horas)": duracao_str,
        "Arquivos Processados": arquivos_processados
    }])
    df_log = pd.concat([df_log, novo_registro], ignore_index=True)
    df_log.to_excel(log_file, index=False)

def listar_arquivos_disponiveis(ano, grupos):
    sia = SIA()
    arquivos_disponiveis = []
    sia.load()  # Load necessary paths or configurations
    for grupo in grupos:
        try:
            print(f"[{datetime.now()}] Tentando carregar arquivos do grupo {grupo} para o ano {ano}.")
            arquivos = sia.get_files(group=grupo, year=ano)
            if arquivos:
                arquivos_disponiveis.extend(arquivos)
                print(f"[{datetime.now()}] Arquivos carregados para {grupo}-{ano}: {arquivos}")
            else:
                print(f"[{datetime.now()}] Nenhum arquivo encontrado para {grupo}-{ano}.")
        except Exception as e:
            print(f"[{datetime.now()}] Erro ao carregar arquivos para {grupo}-{ano}: {e}")
    return arquivos_disponiveis

def processar_e_salvar(arquivo):
    sia = SIA()
    try:
        parquet_file_path = os.path.join(output_dir, f"{arquivo}.parquet")
        
        # Verifica se o arquivo já foi baixado
        if os.path.exists(parquet_file_path):
            print(f"[{datetime.now()}] {arquivo} já foi baixado. Pulando download.")
            return True

        print(f"[{datetime.now()}] Baixando e processando {arquivo}.")
        parquet_set = sia.download([arquivo])
        
        if not parquet_set:
            print(f"[{datetime.now()}] Não foi possível baixar o arquivo {arquivo}.")
            return False

        # Assuming download method also saves the file, otherwise additional saving code would be required
        print(f"[{datetime.now()}] Arquivo {arquivo} salvo em {parquet_file_path}.")
        gc.collect()
        return True
        
    except Exception as e:
        print(f"[{datetime.now()}] Erro ao processar {arquivo}: {e}")
        return False

def main():
    print(f"[{datetime.now()}] Processo iniciado.")
    df_log = iniciar_log()
    inicio = datetime.now()
    
    arquivos_disponiveis = listar_arquivos_disponiveis(ano, grupos)
    total_arquivos = len(arquivos_disponiveis)
    print(f"[{datetime.now()}] Arquivos listados: {total_arquivos} disponíveis.")
    
    arquivos_processados = 0
    for i, arquivo in enumerate(arquivos_disponiveis, 1):
        sucesso = processar_e_salvar(arquivo)
        if sucesso:
            arquivos_processados += 1
        print(f"Arquivos processados: {arquivos_processados} de {total_arquivos}")
        percentual_processado = (arquivos_processados / total_arquivos) * 100
        print(f"Percentual de processamento: {percentual_processado:.0f}%")

    fim = datetime.now()
    print(f"[{datetime.now()}] Processo finalizado.")
    registrar_log(df_log, inicio, fim, arquivos_processados)

if __name__ == "__main__":
    main()


[2024-09-09 15:26:54.492409] Processo iniciado.
[2024-09-09 15:26:56.209384] Tentando carregar arquivos do grupo PA para o ano 2023.
[2024-09-09 15:26:56.404408] Arquivos carregados para PA-2023: [PAAC2301.dbc, PAAC2302.dbc, PAAC2303.dbc, PAAC2304.dbc, PAAC2305.dbc, PAAC2306.dbc, PAAC2307.dbc, PAAC2308.dbc, PAAC2309.dbc, PAAC2310.dbc, PAAC2311.dbc, PAAC2312.dbc, PAAL2301.dbc, PAAL2302.dbc, PAAL2303.dbc, PAAL2304.dbc, PAAL2305.dbc, PAAL2306.dbc, PAAL2307.dbc, PAAL2308.dbc, PAAL2309.dbc, PAAL2310.dbc, PAAL2311.dbc, PAAL2312.dbc, PAAM2301.dbc, PAAM2302.dbc, PAAM2303.dbc, PAAM2304.dbc, PAAM2305.dbc, PAAM2306.dbc, PAAM2307.dbc, PAAM2308.dbc, PAAM2309.dbc, PAAM2310.dbc, PAAM2311.dbc, PAAM2312.dbc, PAAP2301.dbc, PAAP2302.dbc, PAAP2303.dbc, PAAP2304.dbc, PAAP2305.dbc, PAAP2306.dbc, PAAP2307.dbc, PAAP2308.dbc, PAAP2309.dbc, PAAP2310.dbc, PAAP2311.dbc, PAAP2312.dbc, PABA2301.dbc, PABA2302.dbc, PABA2303.dbc, PABA2304.dbc, PABA2305.dbc, PABA2306.dbc, PABA2307.dbc, PABA2308.dbc, PABA2309.dbc, PABA2

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.85M/2.85M [00:00<00:00, 1.25GB/s]


[2024-09-09 15:26:56.466368] Arquivo PAAC2301.dbc salvo em /home/amcjr/pysus/PAAC2301.dbc.parquet.
Arquivos processados: 1 de 372
Percentual de processamento: 0%
[2024-09-09 15:26:56.612756] Baixando e processando PAAC2302.dbc.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.34M/3.34M [00:00<00:00, 1.41GB/s]

[2024-09-09 15:26:56.618520] Arquivo PAAC2302.dbc salvo em /home/amcjr/pysus/PAAC2302.dbc.parquet.
Arquivos processados: 2 de 372
Percentual de processamento: 1%


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7.51M/7.51M [00:00<00:00, 3.72GB/s]

[2024-09-09 15:27:51.677787] Arquivo PATO2311.dbc salvo em /home/amcjr/pysus/PATO2311.dbc.parquet.


Arquivos processados: 371 de 372
Percentual de processamento: 100%
[2024-09-09 15:27:51.798217] Baixando e processando PATO2312.dbc.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7.62M/7.62M [00:00<00:00, 4.20GB/s]


[2024-09-09 15:27:51.803064] Arquivo PATO2312.dbc salvo em /home/amcjr/pysus/PATO2312.dbc.parquet.
Arquivos processados: 372 de 372
Percentual de processamento: 100%
[2024-09-09 15:27:51.874745] Processo finalizado.
